In [1]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt

from pydicom import dcmread
from pydicom.data import get_testdata_file

from tensorflow import keras
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers
from tensorflow.keras import Model
from sklearn.metrics import roc_auc_score
from tensorflow.keras.metrics import AUC
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import cv2

import os
import sys
import gc

ModuleNotFoundError: No module named 'pydicom'

In [ ]:
root_data_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'
train_images_path = root_data_path + '/train'
test_images_path = root_data_path + '/test'
train_labels_path = root_data_path + '/train_labels.csv'

In [13]:
IMAGE_SIZE = 128

In [15]:
def load_data_for_patient(p_id, label, image_type, resize=True, size=256):
    
    image_data = []
    image_labels = []
    files = os.listdir(os.path.join(train_images_path, p_id, image_type))
    
    for f in range(0, len(files), 3):
        dicom_image = dcmread(os.path.join(train_images_path, p_id, image_type, files[f]))
        pixel_array = dicom_image.pixel_array[80:400, 100:420]
        
            
        if np.max(pixel_array) == 0: #Blank image
            continue 
        else:

            if(resize == True):
                pixel_array = cv2.resize(pixel_array, (size, size))
                
            pixel_array = np.array(pixel_array, dtype = float)
            pixel_array /= 4095
            pixel_array = list(pixel_array)
            
            image_data.append(pixel_array)
            image_labels.append(label)
    
    image_data_array = np.array(image_data)
    image_labels_array = np.array(image_labels)
    del files
    del image_data
    del image_labels
    gc.collect()
    #print('Max Pixel Value: ', max_p)
    print(image_data_array.shape)
    return image_data_array, image_labels_array

In [16]:
def load_train_data(image_type, image_size):
    train_data = np.empty((0,image_size,image_size), dtype=float)
    train_labels = np.empty((0), dtype=int)
    train_labels_df =  pd.read_csv(train_labels_path, dtype={'BraTS21ID': str})
    
    for index, patient in train_labels_df.iterrows():
        image_data, image_labels = load_data_for_patient(patient['BraTS21ID'], patient['MGMT_value'], image_type, resize=True, size=image_size)
        if(image_data.shape[0] > 0):
            train_data = np.append(train_data, image_data, axis=0)
            train_labels = np.append(train_labels, image_labels, axis=0)
            
        del image_data
        del image_labels
        gc.collect()
        
    print(train_data.shape)
    print(train_labels.shape)
    return train_data, train_labels
    

In [17]:
train_data, train_labels = load_train_data('FLAIR', IMAGE_SIZE)
X_train, X_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=42)

In [ ]:
#ran - 0.9954, 0.8604
def get_model(image_size):
    img_input = keras.Input(shape=(image_size, image_size, 1))
    
    x = layers.Conv2D(filters = 16, kernel_size=3, activation='relu')(img_input)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.Conv2D(filters = 32, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.Conv2D(filters = 64, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.Conv2D(filters = 128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = Model(img_input, output)
    
    return model


model = get_model(IMAGE_SIZE)
model.summary()

In [ ]:
#ran - 0.9898, 0.8517
def get_model(image_size):
    img_input = keras.Input(shape=(image_size, image_size, 1))
    
    x = layers.Conv2D(filters = 32, kernel_size=3, activation='relu')(img_input)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.01)(x)
    
    x = layers.Conv2D(filters = 128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.03)(x)
    
    x = layers.Conv2D(filters = 128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.04)(x)
    
    x = layers.Conv2D(filters = 128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.03)(x)
    
    x = layers.Conv2D(filters = 128, kernel_size=3, activation='relu')(x)
    x = layers.MaxPooling2D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.04)(x)
    
    x = layers.Flatten()(x)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    
    output = layers.Dense(1, activation='sigmoid')(x)
    
    model = Model(img_input, output)
    
    return model


model = get_model(IMAGE_SIZE)
model.summary()

In [ ]:
print(X_train.shape)

In [ ]:
import gc
gc.collect()
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=[AUC(name='auc'),"acc"],
)
epochs = 100
gc.collect()
model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    shuffle=True,
)